# Часть 2. Подготовка и обучение модели на базовых и обучающих данных

**Описание действий:**

У нас имеется запрос - вектор query (1x72). Для него у нас есть target base-xxx - это вектор из базы base. 
С этим вектором мы идем в faiss, где получаем с его помощью получаем еще несколько таких же векторов (в нашем случае мы взяли 30 шт). На этих данных мы обучаем классификатор. Для каждого вектора, который нам выдал faiss, мы знаем имя base-1, base-2 ...base-xxx. И какой-то из этих векторов и является искомым base-xxx, который прописан у нас в target. 

Следующим шагом мы фиксируем колонку в с таргетом: смотрим на первый вектор из faiss и сравниваем его с target base-xxx. Если значения не совпали, ставим 0, а если совпали,то ставим 1. То же самое делаем для всех векторов. Таким образом мы получаем новый target, на котоом будем обучать CatBoostClassifier. 

Мы берем наш исходный запрос query (1x72) и приставляем к каждому вектору, который нам выдал faiss. В результате мы получаем матрицу 30х144 (где 30 - количество кандидатов, 144 = 72 query + 72 base, по-простому, query и base имеют 72 колонки в датасете). Эта матрица, и будет нашим новым признаком  features (30х144). А столбец с 0 и 1, который мы сформировали, и будет нашим новым таргетом, на котором мы обучаем CatBoostClassifier.

### Загрузка библиотек и исходных датасетов 

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

import faiss

from sklearn.preprocessing import StandardScaler

from catboost import CatBoostClassifier

In [2]:
df_base = pd.read_csv("base.csv", index_col=0)
df_train = pd.read_csv("train.csv", index_col=0)

###  Подготовка данных для работы Faiss

In [3]:
# Разобьем обечающий датасет на фичи и таргет 
targets = df_train["Target"]
df_train.drop("Target", axis=1, inplace=True)

In [4]:
# Нормализация данных
def normalize_with_indices(df):
       
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    
    df_normalized = pd.DataFrame(df_scaled, index=df.index, columns=df.columns)
    return df_normalized

In [5]:
df_base_scaled = normalize_with_indices(df_base)
df_train_scaled  = normalize_with_indices(df_train)

### Первая стадия поиска ближайших соседей с использованием Faiss

In [6]:
# Инициализация Faiss
dims = df_base_scaled.shape[1]
# n_cells = 10
n_cells = 1000

quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

# idx_l2.nprobe = 5
idx_l2.nprobe = 100

In [7]:
# Обучение Faiss на base данных
idx_l2.train(np.ascontiguousarray(df_base_scaled).astype('float32'))

idx_l2.add(np.ascontiguousarray(df_base_scaled).astype('float32'))

In [8]:
base_index = {k: v for k, v in enumerate(df_base_scaled.index.to_list())}

In [9]:
%%time
# Поиск ближайших соседей на тренировочных данных
k_neighbours = 30
vecs, idx = idx_l2.search(np.ascontiguousarray(df_train_scaled.values).astype('float32'), k_neighbours)

CPU times: user 45min 14s, sys: 945 ms, total: 45min 15s
Wall time: 5min 42s


In [10]:
# Вычисляем точность (accuracy) модели на тренировочных данных
acc = 0
for target, el in zip(targets.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(idx))

73.855


### Подготовка данных для обучения CatBoost

In [11]:
# Преобразование индексов в массив numpy
idx = np.array([[base_index[_] for _ in el] for el in idx], dtype=object) 

# Изменение формы расстояний для корректной работы
vecs = vecs.reshape(-1, 1)

In [12]:
# Подготовка целевых значений для использования с CatBoost
cat_target = np.array([[1 if x == target else 0 for x in el] for el, target in zip(idx, targets.values.tolist())]).flatten()

In [13]:
# Извлечение признаков для кандидатов и объектов
candidate_features = df_base_scaled.loc[idx.flatten()].values  # Признаки для кандидатов
object_features = df_train_scaled.values  # Признаки для объектов

In [14]:
del df_base
del df_train
del df_base_scaled 
del df_train_scaled

In [15]:
# Повторение признаков объектов для кандидатов
repeated_object = np.repeat(object_features, k_neighbours, axis=0)

In [16]:
# Горизонтальное объединение массивов
cat_features = np.hstack((vecs, candidate_features, repeated_object))

In [17]:
del idx 
del vecs 
del candidate_features 

### Подготовка и обучение CatBoostClassifier на тренировочном датасете 

In [18]:
# Определяем параметры для CatBoost
params = {
    'n_estimators': 100, 
    'depth': 5,
    'learning_rate': 0.1,
    'eval_metric': 'Accuracy',
    'verbose': 10
}

In [19]:
# Создаем модель CatBoost
cat_model = CatBoostClassifier(**params)

In [20]:
%%time
# Обучаем модель на тренировочных данных 
cat_model.fit(cat_features, cat_target)
# # Ограничим количество данных для обучения, чтобы уменьшить нагрузку на память
# sample_size = 1000  
# cat_model.fit(cat_features[:sample_size], cat_target[:sample_size])

0:	learn: 0.9786183	total: 591ms	remaining: 58.5s
10:	learn: 0.9804710	total: 6.32s	remaining: 51.1s
20:	learn: 0.9810133	total: 12.3s	remaining: 46.3s
30:	learn: 0.9813800	total: 18.4s	remaining: 41.1s
40:	learn: 0.9817200	total: 24.4s	remaining: 35.1s
50:	learn: 0.9819623	total: 30.2s	remaining: 29s
60:	learn: 0.9823613	total: 35.9s	remaining: 22.9s
70:	learn: 0.9826997	total: 41.7s	remaining: 17s
80:	learn: 0.9828950	total: 47.6s	remaining: 11.2s
90:	learn: 0.9831443	total: 53.8s	remaining: 5.32s
99:	learn: 0.9833803	total: 59.2s	remaining: 0us
CPU times: user 10min, sys: 2.92 s, total: 10min 3s
Wall time: 2min 55s


In [21]:
# Сохраняем обученную модель 
cat_model.save_model("cat_model.cbm")

## ВЫВОД

Подготовлена и обучена моель рекомендательной системы (поиска ближайших соседей). 
Поиск осуществляется в две стадии: 
1. Сначала из базы (base) с помощью библиотеки faiss осуществлен поиск ближайших соседей (30 шт) на тренировочной выборке. 
2. Затем с помощью CatBoostClassifier упорядочены и выбираны лучшие результаты.
Обученная модель сохранена для использования на валидационной выборке. 